# Big Brother Brasil 26 - Reaction Analysis Dashboard## OverviewThis notebook analyzes **participant reaction data** from Big Brother Brasil 26 (BBB26), using data from the GloboPlay API. Participants react to each other daily using emoji-based reactions, which reveal social dynamics, alliances, and conflicts within the house.### What are Reactions?Every day, BBB participants can give reactions to other houseguests using emojis. These reactions fall into three categories:| Category | Reactions | Meaning ||----------|-----------|---------|| **Positive** ❤️ | Coração (Heart) | Love, support, friendship || **Mild Negative** 💼🌱🍪 | Mala (Baggage), Planta (Plant), Biscoito (Cookie) | Annoying, invisible/boring, attention-seeking || **Strong Negative** 🐍🎯🤮🤥💔 | Cobra (Snake), Alvo (Target), Vômito (Vomit), Mentiroso (Liar), Coração Partido (Broken Heart) | Betrayal, enemy, disgust, dishonesty, disappointment |### Key Metrics Analyzed1. **Sentiment Score**: Net positivity (positive reactions minus weighted negatives)2. **Controversy Score**: Product of positive and negative reactions (high = divisive)3. **Balance Correlation**: Relationship between game balance (currency) and social standing4. **Alliance Mapping**: Who supports whom, and mutual relationships### Data Source- **API**: `https://apis-globoplay.globo.com/mve-api/globo-play/realities/bbb/participants/`- **Historical Data**: JSON snapshots saved with timestamps for timeline analysis

---## 1. Setup & ConfigurationImport required libraries for data processing, analysis, and visualization.

In [ ]:
# ============================================================================# IMPORTS# ============================================================================import requestsimport jsonimport pandas as pdimport numpy as npimport osimport globfrom datetime import datetimefrom collections import defaultdict# Visualizationimport matplotlib.pyplot as pltimport seaborn as snsimport plotly.graph_objects as goimport plotly.express as pxfrom plotly.subplots import make_subplotsimport networkx as nx# Set style for matplotlibtry:    plt.style.use('seaborn-v0_8-darkgrid')except:    try:        plt.style.use('seaborn-darkgrid')    except:        plt.style.use('dark_background')sns.set_palette("husl")print("✅ All libraries imported successfully!")

---## 2. Reaction CategorizationDefine the sentiment categories for each reaction type. This categorization is crucial for all analysis.### Why These Categories?- **Positive**: Only the heart emoji represents genuine support- **Mild Negative**: These suggest annoyance but not conflict  - 🌱 Planta (Plant) = "You don't do anything, just blend in"  - 💼 Mala (Baggage) = "You're annoying/a burden"  - 🍪 Biscoito (Cookie) = "You're just seeking attention"- **Strong Negative**: These indicate real conflict or betrayal  - 🐍 Cobra (Snake) = "You're a backstabber"  - 🎯 Alvo (Target) = "You're my target/enemy"  - 🤮 Vômito (Vomit) = "I'm disgusted by you"  - 🤥 Mentiroso (Liar) = "You're dishonest"  - 💔 Coração Partido (Broken Heart) = "You disappointed me" 

In [ ]:
# ============================================================================# REACTION CATEGORIES & EMOJI MAPPING# ============================================================================# Emoji mapping for displayEMOJI_MAP = {    'Coração': '❤️',    'Coração partido': '💔',    'Cobra': '🐍',    'Mala': '💼',    'Planta': '🌱',    'Biscoito': '🍪',    'Alvo': '🎯',    'Vômito': '🤮',    'Mentiroso': '🤥'}# Sentiment categoriesPOSITIVE_REACTIONS = ['Coração']MILD_NEGATIVE_REACTIONS = ['Planta', 'Mala', 'Biscoito']STRONG_NEGATIVE_REACTIONS = ['Cobra', 'Alvo', 'Vômito', 'Mentiroso', 'Coração partido']# Sentiment weights for scoringSENTIMENT_WEIGHTS = {    'positive': 1.0,      # Full positive weight    'mild_negative': -0.5,  # Half negative weight (less severe)    'strong_negative': -1.0  # Full negative weight}def get_reaction_emoji(reaction_name):    """Return emoji for a Portuguese reaction name"""    return EMOJI_MAP.get(reaction_name, reaction_name)def categorize_reaction(reaction_name):    """Categorize a reaction into sentiment categories"""    if reaction_name in POSITIVE_REACTIONS:        return 'positive'    elif reaction_name in MILD_NEGATIVE_REACTIONS:        return 'mild_negative'    elif reaction_name in STRONG_NEGATIVE_REACTIONS:        return 'strong_negative'    return 'unknown'def get_sentiment_weight(reaction_name):    """Get numerical sentiment weight for a reaction"""    category = categorize_reaction(reaction_name)    return SENTIMENT_WEIGHTS.get(category, 0)print("✅ Reaction categories configured:")print(f"   Positive: {[f'{r} {get_reaction_emoji(r)}' for r in POSITIVE_REACTIONS]}")print(f"   Mild Negative: {[f'{r} {get_reaction_emoji(r)}' for r in MILD_NEGATIVE_REACTIONS]}")print(f"   Strong Negative: {[f'{r} {get_reaction_emoji(r)}' for r in STRONG_NEGATIVE_REACTIONS]}")

---## 3. Data CollectionFunctions to fetch data from the GloboPlay API and manage local backups.### Data Flow1. **Primary**: Fetch fresh data from API2. **Backup**: Save timestamped JSON files locally  3. **Fallback**: Load most recent local file if API fails### Historical DataEach run saves a new JSON file with timestamp, enabling timeline analysis of how reactions change over the season.

In [ ]:
# ============================================================================# DATA COLLECTION FUNCTIONS# ============================================================================API_URL = "https://apis-globoplay.globo.com/mve-api/globo-play/realities/bbb/participants/"def save_api_response(api_url=API_URL):    """Fetch data from API and save with timestamp"""    try:        response = requests.get(api_url, timeout=10)        response.raise_for_status()        data = response.json()                # Save with timestamp        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")        filename = f"bbb_participants_{timestamp}.json"                with open(filename, 'w', encoding='utf-8') as f:            json.dump(data, f, indent=2, ensure_ascii=False)                print(f"✅ Data saved to {filename}")        print(f"   Participants found: {len(data)}")        return data            except requests.RequestException as e:        print(f"⚠️ API request failed: {e}")        return Nonedef load_latest_saved_data():    """Load the most recent saved JSON file as fallback"""    json_files = sorted(glob.glob("bbb_participants_*.json"))        if not json_files:        print("❌ No saved data files found")        return None        latest_file = json_files[-1]    print(f"📂 Loading: {latest_file}")        with open(latest_file, 'r', encoding='utf-8') as f:        data = json.load(f)        print(f"   Participants loaded: {len(data)}")    return datadef get_all_reaction_types(data):    """Extract all unique reaction types from data"""    reactions = set()    for participant in data:        received = participant.get('characteristics', {}).get('receivedReactions', [])        for reaction in received:            if label := reaction.get('label'):                reactions.add(label)    return sorted(reactions)print("✅ Data collection functions ready")

---## 4. Data ProcessingTransform raw API data into structured formats for analysis.### Data Structures Created1. **organized_data**: List of processed participant dictionaries2. **cross_table**: Pandas DataFrame showing reaction counts between all pairs3. **sentiment_matrix**: Numerical sentiment scores between participants

In [ ]:
# ============================================================================# DATA PROCESSING FUNCTIONS# ============================================================================def process_participants(data, verbose=False):    """Process raw API data into organized participant list        Returns:        List of dicts with participant info and their received reactions    """    if not data:        print("❌ No data to process")        return []    organized_data = []    for participant in data:        chars = participant.get('characteristics', {})                participant_data = {            "name": participant.get('name', 'N/A'),            "job": chars.get('job', 'N/A'),            "group": chars.get('group', 'N/A'),        # Vip or Xepa            "memberOf": chars.get('memberOf', 'N/A'),  # Camarote, Pipoca, Veterano            "balance": chars.get('balance', 0),            "eliminated": chars.get('eliminated', False),            "roles": [role.get('label', 'N/A') for role in chars.get('roles', [])],            "reactions": {},            "total_reactions": 0,            "duo_partner": (participant.get('duo') or {}).get('name', 'None')        }        # Process received reactions        received_reactions = chars.get('receivedReactions', [])        for reaction in received_reactions:            label = reaction.get('label')            amount = reaction.get('amount', 0)            givers = [p.get('name') for p in reaction.get('participants', [])]                        participant_data['reactions'][label] = {                'amount': amount,                'givers': givers            }            participant_data['total_reactions'] += amount                organized_data.append(participant_data)                if verbose:            print(f"{participant_data['name']}: {participant_data['total_reactions']} reactions")    print(f"✅ Processed {len(organized_data)} participants")    return organized_datadef create_cross_table(organized_data):    """Create reaction matrix: who reacted to whom        Returns:        DataFrame with givers as rows, receivers as columns    """    # Get all unique names    all_names = set()    for p in organized_data:        all_names.add(p['name'])        for reaction_data in p['reactions'].values():            all_names.update(reaction_data['givers'])        all_names = sorted(all_names)        # Initialize matrix    matrix = pd.DataFrame(0, index=all_names, columns=all_names)        # Fill in reactions    for p in organized_data:        receiver = p['name']        for reaction_name, reaction_data in p['reactions'].items():            for giver in reaction_data['givers']:                if giver in matrix.index and receiver in matrix.columns:                    matrix.loc[giver, receiver] += 1        print(f"✅ Cross table created: {len(all_names)}x{len(all_names)}")    return matrixdef create_sentiment_matrix(organized_data):    """Create weighted sentiment matrix        Positive reactions = +1, Mild negative = -0.5, Strong negative = -1    """    all_names = sorted(set(p['name'] for p in organized_data))    matrix = pd.DataFrame(0.0, index=all_names, columns=all_names)        for p in organized_data:        receiver = p['name']        for reaction_name, reaction_data in p['reactions'].items():            weight = get_sentiment_weight(reaction_name)            for giver in reaction_data['givers']:                if giver in matrix.index:                    matrix.loc[giver, receiver] += weight        return matrixprint("✅ Data processing functions ready")

---## 5. Analysis FunctionsCalculate metrics and generate insights from the reaction data.### Key Metrics| Metric | Formula | Meaning ||--------|---------|---------|| **Sentiment Score** | positive - 0.5×mild - 1×strong | Net positivity received || **Controversy Score** | positive × (mild + strong) | How divisive someone is || **Positive %** | positive / total × 100 | % of reactions that are positive |

In [ ]:
# ============================================================================# ANALYSIS FUNCTIONS# ============================================================================def analyze_reactions(organized_data):    """Calculate reaction statistics for each participant"""    analysis = {}        for p in organized_data:        name = p['name']        total = p.get('total_reactions', 0)                positive = 0        mild_negative = 0        strong_negative = 0                for reaction_name, data in p['reactions'].items():            amount = data.get('amount', 0)            category = categorize_reaction(reaction_name)                        if category == 'positive':                positive += amount            elif category == 'mild_negative':                mild_negative += amount            elif category == 'strong_negative':                strong_negative += amount                # Calculate metrics        sentiment = positive - 0.5*mild_negative - strong_negative        controversy = positive * (mild_negative + strong_negative)                analysis[name] = {            'total': total,            'positive': positive,            'mild_negative': mild_negative,            'strong_negative': strong_negative,            'positive_pct': (positive / total * 100) if total > 0 else 0,            'sentiment_score': sentiment,            'controversy_score': controversy,            'balance': p.get('balance', 0),            'group': p.get('group', 'N/A'),            'memberOf': p.get('memberOf', 'N/A')        }        return analysisdef find_alliances_and_enemies(organized_data):    """Identify mutual relationships between participants"""    sentiment_given = defaultdict(lambda: defaultdict(float))        for p in organized_data:        receiver = p['name']        for reaction_name, data in p['reactions'].items():            weight = get_sentiment_weight(reaction_name)            for giver in data['givers']:                sentiment_given[giver][receiver] += weight        # Find mutual relationships    relationships = []    names = sorted(set(p['name'] for p in organized_data))    checked = set()        for p1 in names:        for p2 in names:            if p1 < p2 and (p1, p2) not in checked:                checked.add((p1, p2))                s1_to_2 = sentiment_given[p1].get(p2, 0)                s2_to_1 = sentiment_given[p2].get(p1, 0)                                if s1_to_2 != 0 or s2_to_1 != 0:                    if s1_to_2 > 0 and s2_to_1 > 0:                        rel_type = 'Allies'                    elif s1_to_2 < 0 and s2_to_1 < 0:                        rel_type = 'Enemies'                    else:                        rel_type = 'Mixed'                                        relationships.append({                        'person1': p1,                        'person2': p2,                        'p1_to_p2': s1_to_2,                        'p2_to_p1': s2_to_1,                        'mutual_sentiment': s1_to_2 + s2_to_1,                        'relationship': rel_type                    })        return pd.DataFrame(relationships)def print_analysis_report(analysis):    """Print comprehensive analysis report"""    df = pd.DataFrame(analysis).T    df = df.sort_values('sentiment_score', ascending=False)        print("\n" + "="*70)    print("📊 REACTION ANALYSIS REPORT")    print("="*70)        print("\n🏆 TOP 5 - Most Loved (Highest Sentiment):")    print("-"*50)    for name in df.head(5).index:        data = analysis[name]        print(f"  {name}: {data['sentiment_score']:.1f} ({data['positive_pct']:.0f}% positive)")        print("\n⚠️ BOTTOM 5 - Most Targeted (Lowest Sentiment):")    print("-"*50)    for name in df.tail(5).index:        data = analysis[name]        print(f"  {name}: {data['sentiment_score']:.1f} ({data['positive_pct']:.0f}% positive)")        print("\n🎭 TOP 5 - Most Controversial (Divisive):")    print("-"*50)    controversial = df.nlargest(5, 'controversy_score')    for name in controversial.index:        data = analysis[name]        print(f"  {name}: controversy={data['controversy_score']:.0f} (❤️{data['positive']} vs 👎{data['mild_negative']+data['strong_negative']})")        return dfprint("✅ Analysis functions ready")

---## 6. VisualizationsCreate charts and graphs to visualize reaction patterns.### Visualizations Generated1. **Sentiment Heatmap**: Color-coded matrix of who likes/dislikes whom2. **Balance Correlation**: Scatter plot of game balance vs sentiment3. **Timeline Analysis**: How reactions change over time (using historical data)4. **Network Graph**: Visual network of relationships5. **Analysis Dashboard**: Multi-panel summary charts

In [ ]:
# ============================================================================# VISUALIZATION FUNCTIONS# ============================================================================def create_sentiment_heatmap(organized_data, figsize=(14, 12)):    """Create heatmap showing sentiment between all participants"""    sentiment_matrix = create_sentiment_matrix(organized_data)        plt.figure(figsize=figsize)        # Create custom colormap (red for negative, white for neutral, green for positive)    cmap = sns.diverging_palette(10, 130, as_cmap=True)        sns.heatmap(        sentiment_matrix,        annot=True,        fmt='.1f',        cmap=cmap,        center=0,        linewidths=0.5,        square=True,        cbar_kws={'label': 'Sentiment (+ = positive, - = negative)'}    )        plt.title('Reaction Sentiment Matrix\n(Rows = Giver, Columns = Receiver)', fontsize=14)    plt.xlabel('Received Reactions From')    plt.ylabel('Gave Reactions To')    plt.xticks(rotation=45, ha='right')    plt.yticks(rotation=0)    plt.tight_layout()        # Save    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")    filename = f"reaction_heatmap_{timestamp}.png"    plt.savefig(filename, dpi=150, bbox_inches='tight')    print(f"📊 Saved: {filename}")        plt.show()def create_balance_correlation(organized_data, analysis):    """Scatter plot: Game balance vs Sentiment score"""    data = []    for p in organized_data:        name = p['name']        if name in analysis:            data.append({                'name': name,                'balance': p.get('balance', 0),                'sentiment': analysis[name]['sentiment_score'],                'group': p.get('group', 'Unknown'),                'type': p.get('memberOf', 'Unknown')            })        df = pd.DataFrame(data)        fig, axes = plt.subplots(1, 2, figsize=(14, 6))        # Plot by group (Vip/Xepa)    colors_group = {'Vip': '#FFD700', 'Xepa': '#87CEEB'}    for group in df['group'].unique():        mask = df['group'] == group        axes[0].scatter(            df[mask]['balance'],             df[mask]['sentiment'],            c=colors_group.get(group, 'gray'),            label=group,            s=100,            alpha=0.7        )        # Add labels    for _, row in df.iterrows():        axes[0].annotate(            row['name'].split()[0],  # First name only            (row['balance'], row['sentiment']),            fontsize=8,            alpha=0.7        )        axes[0].set_xlabel('Game Balance (BBBs)')    axes[0].set_ylabel('Sentiment Score')    axes[0].set_title('Balance vs Sentiment by Group')    axes[0].legend()    axes[0].grid(True, alpha=0.3)        # Plot by type (Camarote/Pipoca/Veterano)    colors_type = {'Camarote': '#9B59B6', 'Pipoca': '#E74C3C', 'Veterano': '#2ECC71'}    for ptype in df['type'].unique():        mask = df['type'] == ptype        axes[1].scatter(            df[mask]['balance'],             df[mask]['sentiment'],            c=colors_type.get(ptype, 'gray'),            label=ptype,            s=100,            alpha=0.7        )        for _, row in df.iterrows():        axes[1].annotate(            row['name'].split()[0],            (row['balance'], row['sentiment']),            fontsize=8,            alpha=0.7        )        axes[1].set_xlabel('Game Balance (BBBs)')    axes[1].set_ylabel('Sentiment Score')    axes[1].set_title('Balance vs Sentiment by Type')    axes[1].legend()    axes[1].grid(True, alpha=0.3)        plt.tight_layout()        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")    filename = f"balance_correlation_{timestamp}.png"    plt.savefig(filename, dpi=150, bbox_inches='tight')    print(f"📊 Saved: {filename}")        plt.show()        # Calculate correlation    corr = df['balance'].corr(df['sentiment'])    print(f"\n📈 Correlation (Balance vs Sentiment): {corr:.3f}")print("✅ Visualization functions ready")

In [ ]:
def create_reaction_network(organized_data, min_sentiment=0.5):    """Create network graph of relationships"""    import networkx as nx        sentiment_matrix = create_sentiment_matrix(organized_data)        G = nx.DiGraph()        # Add nodes    for p in organized_data:        G.add_node(p['name'],                    group=p.get('group', 'Unknown'),                   balance=p.get('balance', 0))        # Add edges for significant relationships    for giver in sentiment_matrix.index:        for receiver in sentiment_matrix.columns:            if giver != receiver:                sentiment = sentiment_matrix.loc[giver, receiver]                if abs(sentiment) >= min_sentiment:                    G.add_edge(giver, receiver,                               weight=abs(sentiment),                              sentiment=sentiment)        plt.figure(figsize=(16, 12))        # Position nodes    pos = nx.spring_layout(G, k=2, iterations=50, seed=42)        # Color nodes by group    node_colors = ['#FFD700' if G.nodes[n].get('group') == 'Vip' else '#87CEEB'                    for n in G.nodes()]        # Draw nodes    nx.draw_networkx_nodes(G, pos, node_color=node_colors,                            node_size=1500, alpha=0.8)    nx.draw_networkx_labels(G, pos, font_size=8)        # Draw edges - green for positive, red for negative    positive_edges = [(u, v) for u, v, d in G.edges(data=True) if d['sentiment'] > 0]    negative_edges = [(u, v) for u, v, d in G.edges(data=True) if d['sentiment'] < 0]        nx.draw_networkx_edges(G, pos, edgelist=positive_edges,                           edge_color='green', alpha=0.5,                            arrows=True, arrowsize=15)    nx.draw_networkx_edges(G, pos, edgelist=negative_edges,                           edge_color='red', alpha=0.5,                            arrows=True, arrowsize=15)        plt.title('Reaction Network\n(Green = Positive, Red = Negative)', fontsize=14)    plt.axis('off')        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")    filename = f"reaction_network_{timestamp}.png"    plt.savefig(filename, dpi=150, bbox_inches='tight')    print(f"📊 Saved: {filename}")        plt.show()def analyze_reaction_timeline():    """Analyze how reactions change over time using saved JSON files"""    json_files = sorted(glob.glob("bbb_participants_*.json"))        if len(json_files) < 2:        print("⚠️ Need at least 2 data files for timeline analysis")        return        # Get unique dates (deduplicate same-day multiple runs)    daily_files = {}    for f in json_files:        date = f.split('_')[2]  # Extract date from filename        daily_files[date] = f  # Keep latest file per day        print(f"📅 Found {len(daily_files)} unique days of data")        timeline_data = []        for date, filepath in sorted(daily_files.items()):        with open(filepath, 'r') as f:            data = json.load(f)                for p in data:            name = p['name']            chars = p.get('characteristics', {})            received = chars.get('receivedReactions', [])                        pos = sum(r['amount'] for r in received if r['label'] in POSITIVE_REACTIONS)            neg = sum(r['amount'] for r in received if r['label'] in MILD_NEGATIVE_REACTIONS + STRONG_NEGATIVE_REACTIONS)                        timeline_data.append({                'date': date,                'name': name,                'positive': pos,                'negative': neg,                'sentiment': pos - neg            })        df = pd.DataFrame(timeline_data)        # Plot sentiment over time for top/bottom participants    latest_analysis = df[df['date'] == max(df['date'])]    top_5 = latest_analysis.nlargest(5, 'sentiment')['name'].tolist()    bottom_5 = latest_analysis.nsmallest(5, 'sentiment')['name'].tolist()        fig, axes = plt.subplots(2, 1, figsize=(14, 10))        for name in top_5:        person_data = df[df['name'] == name].sort_values('date')        axes[0].plot(person_data['date'], person_data['sentiment'],                     marker='o', label=name)        axes[0].set_title('Sentiment Over Time - Most Loved')    axes[0].legend(bbox_to_anchor=(1.05, 1))    axes[0].tick_params(axis='x', rotation=45)        for name in bottom_5:        person_data = df[df['name'] == name].sort_values('date')        axes[1].plot(person_data['date'], person_data['sentiment'],                     marker='o', label=name)        axes[1].set_title('Sentiment Over Time - Most Targeted')    axes[1].legend(bbox_to_anchor=(1.05, 1))    axes[1].tick_params(axis='x', rotation=45)        plt.tight_layout()        timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")    filename = f"reaction_timeline_{timestamp}.png"    plt.savefig(filename, dpi=150, bbox_inches='tight')    print(f"📊 Saved: {filename}")        plt.show()print("✅ Additional visualization functions ready")

---## 7. Main ExecutionRun the complete analysis pipeline.

In [ ]:
# ============================================================================# MAIN EXECUTION# ============================================================================def main():    """Main analysis pipeline"""        # 1. Fetch or load data    print("="*70)    print("📥 LOADING DATA")    print("="*70)        data = save_api_response()        if not data:        print("\n⚠️ API failed, using saved data...")        data = load_latest_saved_data()        if not data:        print("❌ No data available")        return        # 2. Show reaction types    print("\n" + "="*70)    print("🎭 REACTION TYPES FOUND")    print("="*70)    for reaction in get_all_reaction_types(data):        emoji = get_reaction_emoji(reaction)        category = categorize_reaction(reaction)        print(f"  {emoji} {reaction} → {category}")        # 3. Process data    print("\n" + "="*70)    print("⚙️ PROCESSING DATA")    print("="*70)    organized_data = process_participants(data)    cross_table = create_cross_table(organized_data)        # 4. Analysis    print("\n" + "="*70)    print("🔍 RUNNING ANALYSIS")    print("="*70)    analysis = analyze_reactions(organized_data)    df_analysis = print_analysis_report(analysis)        # 5. Find relationships    print("\n" + "="*70)    print("🤝 RELATIONSHIP ANALYSIS")    print("="*70)    relationships = find_alliances_and_enemies(organized_data)        if not relationships.empty:        print("\nTop Mutual Allies:")        allies = relationships[relationships['relationship'] == 'Allies'].nlargest(10, 'mutual_sentiment')        for _, row in allies.iterrows():            print(f"  ❤️ {row['person1']} <-> {row['person2']}: {row['mutual_sentiment']:.1f}")                print("\nTop Mutual Enemies:")        enemies = relationships[relationships['relationship'] == 'Enemies'].nsmallest(5, 'mutual_sentiment')        for _, row in enemies.iterrows():            print(f"  ⚔️ {row['person1']} <-> {row['person2']}: {row['mutual_sentiment']:.1f}")        # 6. Visualizations    print("\n" + "="*70)    print("📊 CREATING VISUALIZATIONS")    print("="*70)        create_sentiment_heatmap(organized_data)    create_balance_correlation(organized_data, analysis)    create_reaction_network(organized_data)    analyze_reaction_timeline()        print("\n" + "="*70)    print("✅ ANALYSIS COMPLETE")    print("="*70)        return organized_data, analysis, cross_table, relationships# Run the analysisorganized_data, analysis, cross_table, relationships = main()

---## 8. Cross Table ViewDisplay the full reaction cross-table showing all participant interactions.

In [ ]:
# Display the cross tableprint("\n📋 REACTION CROSS TABLE")print("Rows = Who gave reactions, Columns = Who received them")print("="*70)display(cross_table)# Save cross table to CSVtimestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")csv_filename = f"reaction_cross_table_{timestamp}.csv"cross_table.to_csv(csv_filename)print(f"\n💾 Cross table saved to: {csv_filename}")